# User Interface - Image Classification

In [1]:
import os
import tkinter as tk
from tkinter import *
from tkinter import filedialog, font
from PIL import ImageTk, Image, UnidentifiedImageError
import numpy
from keras.models import load_model

def load_class_names(train_dir):
    return sorted(os.listdir(train_dir))

train_dir = r"C:\Users\admir\UK\United Kingdom\Personal Projects\Image Classification\image-classification\dataset\train-test\train\\"
class_names = load_class_names(train_dir)

THRESHOLD = 0.9

model = load_model(r"C:\Users\admir\UK\United Kingdom\Personal Projects\Image Classification\image-classification\training_model.h5")

class Homepage(tk.Frame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # window sizes
        window_width = 800
        window_height = 600
        
        # screen sizes
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        
        # coordinates to make the screen at the center
        x_coordinate = (screen_width/2) - (window_width/2)
        y_coordinate = (screen_height/2) - (window_height/2)
        
        # window geometry
        self.master.geometry("%dx%d+%d+%d" %(window_width, window_height,
                                            x_coordinate, y_coordinate))
        
        # window frame
        window_frame = Frame(self, bg = '#f8ca9d', width = window_width, height = window_height)
        window_frame.grid(row = 0, column = 0)
        
        # title
        title = Label(window_frame, text = 'Image Classification', fg = 'black', bg = '#f8ca9d')
        title.configure(font = ('arial', 24, 'bold'))
        title.place(relx = 0.5, rely = 0.15, anchor = 'center')
        
        # make underline for the title
        default_font = font.Font(font = title.cget("font"))
        default_font.configure(underline = True)
        underline_font = default_font
        title.config(font = underline_font)
        
        # label
        self.label = Label(window_frame, text = '', fg = 'black', bg = '#f8ca9d')
        self.label.configure(font = ('arial', 18, 'bold'))
        self.label.place(relx = 0.5, rely = 0.3, anchor = 'center')
        
        # img
        self.picture = Label(window_frame, bg = '#f8ca9d')
        self.picture.place(relx = 0.5, rely = 0.5, anchor = 'center')
        
        # classify image button
        classify_button = Button(window_frame, text = 'Classify Image', fg = 'black', bg = '#c5d7c0', command = self.classify_click)
        classify_button.configure(font = ('Gatwick', 12, 'bold'), padx = 10, pady = 10)
        classify_button.place(relx = 0.38, rely = 0.75, anchor = 'center')
        
        # upload image button
        upload_button = Button(window_frame, text = 'Upload Image', fg = 'black',
                              bg = '#c5d7c0', command = self.upload_click)
        upload_button.configure(font = ('Gatwick', 12, 'bold'), padx = 10, pady = 10)
        upload_button.place(relx = 0.62, rely = 0.75, anchor = 'center')
        
        self.image_path = None

    def upload_click(self):
        file_path = filedialog.askopenfilename()
        
        if file_path:
            try:
                self.image_path = file_path
                uploaded = Image.open(file_path)
                uploaded.thumbnail(((self.winfo_width()/3), (self.winfo_height()/3)))
                img = ImageTk.PhotoImage(uploaded)
                self.picture.configure(image = img)
                self.picture.image = img
                self.label.configure(foreground = 'black', text = '')
            except UnidentifiedImageError:
                self.label.configure(text = 'Error!')
                self.picture.configure(image = None)
                self.picture.image = None
                self.image_path = None
            except Exception as e:
                self.label.configure(text = 'Error!')
                self.picture.configure(image = None)
                self.picture.image = None
                self.image_path = None
            
    def classify_click(self):
        global label_packed
        
        if self.image_path:
            image = Image.open(self.image_path)
            image = image.resize((224, 224))
            image = numpy.expand_dims(image, axis = 0)
            image = numpy.array(image)
            result = model.predict(image)
            max_confidence = numpy.max(result)
            
            if max_confidence < THRESHOLD:
                class_label = "Unknown"
                
            else:
                class_idx = numpy.argmax(result, axis = 1)[0]
                class_label = class_names[class_idx]
                
            self.label.configure(foreground = 'black', text = class_label.upper())
            print(result[0])
            
        else:
            self.label.configure(foreground = 'black', text = 'Error!')

def main():
    window = tk.Tk()
    window.title('Image Classification')
    window.configure(background = '#f8ca9d')
    app = Homepage(window)
    app.pack()
    window.mainloop()
    
if __name__ == '__main__':
    main()

1/1 [==============================] - 0s 262ms/step
[1.0000000e+00 1.4484914e-13 1.8162830e-19]
